In [1]:
import pandas
from data_algebra.data_ops import *
import data_algebra.test_util

In [2]:
d = pandas.read_csv('d.csv.gz')

d

,x,g
0,0.376972,level_000357975
1,0.301548,level_000668062
2,-1.098023,level_000593766
3,-1.130406,level_000203296
4,-2.796534,level_000064605
...,...,...
999995,0.154607,level_000029194
999996,-0.241628,level_000721132
999997,0.727351,level_000698435
999998,-1.705844,level_000237171


In [3]:
ops = describe_table(d, table_name='d'). \
    extend({
        'rn': '_row_number()',
        'cs': 'x.cumsum()'
        },
        partition_by=['g'],
        order_by=['x']). \
    order_rows(['g', 'x'])

ops    

TableDescription(
 table_name='d',
 column_names=[
   'x', 'g']) .\
   extend({
    'rn': '_row_number()',
    'cs': 'x.cumsum()'},
   partition_by=['g'],
   order_by=['x']) .\
   order_rows(['g', 'x'])

In [4]:
res = ops.transform(d)

res

,x,g,rn,cs
0,-0.920397,level_000000002,1,-0.920397
1,0.537211,level_000000003,1,0.537211
2,0.734919,level_000000004,1,0.734919
3,-0.890755,level_000000005,1,-0.890755
4,1.702935,level_000000008,1,1.702935
...,...,...,...,...
999995,1.435739,level_000999990,4,1.569817
999996,0.262819,level_000999993,1,0.262819
999997,0.081815,level_000999995,1,0.081815
999998,1.553806,level_000999997,1,1.553806


In [5]:
expect = pandas.read_csv('res.csv.gz')

In [6]:
expect

,x,g,rn,cs
0,-0.920397,level_000000002,1,-0.920397
1,0.537211,level_000000003,1,0.537211
2,0.734919,level_000000004,1,0.734919
3,-0.890755,level_000000005,1,-0.890755
4,1.702935,level_000000008,1,1.702935
...,...,...,...,...
999995,1.435739,level_000999990,4,1.569817
999996,0.262819,level_000999993,1,0.262819
999997,0.081815,level_000999995,1,0.081815
999998,1.553806,level_000999997,1,1.553806


In [7]:
assert data_algebra.test_util.equivalent_frames(res, expect)

In [8]:
time(ops.transform(d))

CPU times: user 11.2 s, sys: 792 ms, total: 11.9 s
Wall time: 12.7 s


,x,g,rn,cs
0,-0.920397,level_000000002,1,-0.920397
1,0.537211,level_000000003,1,0.537211
2,0.734919,level_000000004,1,0.734919
3,-0.890755,level_000000005,1,-0.890755
4,1.702935,level_000000008,1,1.702935
...,...,...,...,...
999995,1.435739,level_000999990,4,1.569817
999996,0.262819,level_000999993,1,0.262819
999997,0.081815,level_000999995,1,0.081815
999998,1.553806,level_000999997,1,1.553806


In [9]:
import timeit 

def f():
    return ops.transform(d)

timeit.timeit(f, number=5)

65.27684559400001

In [10]:
import data_algebra.SQLite

In [11]:
dbmodel = data_algebra.SQLite.SQLiteModel()

In [12]:
print(ops.to_sql(dbmodel, pretty=True))

SELECT "x",
       "g",
       "rn",
       "cs"
FROM
  (SELECT "x",
          "g",
          ROW_NUMBER() OVER (PARTITION BY "g"
                             ORDER BY "x") AS "rn",
                            SUM("x") OVER (PARTITION BY "g"
                                           ORDER BY "x") AS "cs"
   FROM "d") "extend_1"
ORDER BY "g",
         "x"


In [13]:
import sqlite3

In [14]:
conn = sqlite3.connect(':memory:')
dbmodel.prepare_connection(conn)

In [15]:
def f_db():
    try:
        dbmodel.read_query(conn, "DROP TABLE d")
    except:
        pass
    dbmodel.insert_table(conn, d, 'd')
    return dbmodel.read_query(conn, ops.to_sql(dbmodel))

In [16]:
res_db = f_db()

assert data_algebra.test_util.equivalent_frames(res_db, expect)

In [17]:
time(f_db())

CPU times: user 10 s, sys: 656 ms, total: 10.7 s
Wall time: 14.1 s


,x,g,rn,cs
0,-0.920397,level_000000002,1,-0.920397
1,0.537211,level_000000003,1,0.537211
2,0.734919,level_000000004,1,0.734919
3,-0.890755,level_000000005,1,-0.890755
4,1.702935,level_000000008,1,1.702935
...,...,...,...,...
999995,1.435739,level_000999990,4,1.569817
999996,0.262819,level_000999993,1,0.262819
999997,0.081815,level_000999995,1,0.081815
999998,1.553806,level_000999997,1,1.553806


In [18]:
timeit.timeit(f_db, number=5)

73.350337836

In [19]:
# neaten up
conn.close()